In [10]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from collections import deque

# 加载数据文件
data_path = "jq_stock_ticks.csv"
data = pd.read_csv(data_path)

# 定义五档买卖价量数据作为特征
feature_columns = [col for col in data.columns if col.startswith('a') or col.startswith('b')]
features = data[feature_columns].iloc[:-100]

# 计算未来100条数据的成交均价
target_100 = data['current'].rolling(window=101).mean().shift(-100)[:-100]

# 创建并训练梯度提升回归模型
regressor = GradientBoostingRegressor(random_state=42)
regressor.fit(features, target_100)


GradientBoostingRegressor(random_state=42)

In [12]:
from datetime import timedelta

# 加载新数据进行预测
new_data_path = "jq_stock_ticks_next.csv"
new_data_iterator = pd.read_csv(new_data_path, iterator=True, chunksize=1)

# 初始化变量
capital = 100000  # 初始资本
shares = 0  # 持有股票数量
today_purchased_shares = 0
buy_sell_log = []  # 存储买入和卖出的日志
threshold = 0.01  # 买入卖出门槛
trade_count = 0  # 交易次数
prev_date = None
prev_trade_time = None  # 上一次交易时间
current_price = 0
timestamp = ''

# 迭代新数据
for chunk in new_data_iterator:
    chunk['timestamp'] = pd.to_datetime(chunk['time'])
    chunk['date'] = chunk['timestamp'].dt.date

    # 提取新数据中的五档买卖价量数据
    new_features = chunk[feature_columns]
    current_price = chunk['current'].iloc[0]
    date = chunk['date'].iloc[0]
    timestamp = chunk['timestamp'].iloc[0]

    # 检查时间差距
    if prev_trade_time and timestamp - prev_trade_time < timedelta(minutes=1):
        continue  # 若时间差小于一分钟，跳过本次交易

    # 使用训练的模型进行预测
    prediction = regressor.predict(new_features)[0]
    
    price_difference = prediction - current_price

    if prev_date and prev_date != date:
        today_purchased_shares = 0

    if price_difference > threshold and capital > 0:  # 买入条件
        purchase_shares = capital / current_price * (1 - 0.0003)  # 扣除交易费
        today_purchased_shares += purchase_shares
        shares += purchase_shares
        capital = 0
        buy_sell_log.append((timestamp, 'buy', current_price))
        trade_count += 1
        prev_trade_time = timestamp  # 更新上一次交易时间
    elif price_difference < -threshold and shares > today_purchased_shares:  # 卖出条件
        sell_shares = shares - today_purchased_shares  # 只卖出非今天购买的股票
        capital += sell_shares * current_price * (1 - 0.0003)  # 扣除交易费
        shares -= sell_shares
        buy_sell_log.append((timestamp, 'sell', current_price))
        trade_count += 1
        prev_trade_time = timestamp  # 更新上一次交易时间

    prev_date = date

# 最后的卖出（如果还持有股票）
if shares > 0:
    capital += shares * current_price * (1 - 0.0003)
    buy_sell_log.append((timestamp, 'sell', current_price))

# 打印买入和卖出日志
for log in buy_sell_log:
    print(f"Time: {log[0]}, Action: {log[1]}, Price: {log[2]:.2f}")

initial_capital = 100000
final_capital = capital
profit_percentage = (final_capital - initial_capital) / initial_capital * 100

print(f"最终资本: {final_capital:.2f}")
print(f"收益率: {profit_percentage:.2f}%")
print(f"总交易次数: {trade_count}")


[2.0255987]
[2.02534244]
[2.02491466]
[2.02465787]
[2.01829828]
[2.01829828]
[2.01823691]
[2.01823691]
[2.01684165]
[2.01684165]
[2.01684165]
[2.01680778]
[2.01684165]
[2.01636967]
[2.01636967]
[2.01636967]
[2.01636967]
[2.01636967]
[2.01636967]
[2.01636967]
[2.01618623]
[2.01636967]
[2.01636967]
[2.01611341]
[2.01583993]
[2.01583993]
[2.01583993]
[2.01583993]
[2.01583993]
[2.01583993]
[2.01612972]
[2.01568209]
[2.01612972]
[2.01583993]
[2.01583993]
[2.01583993]
[2.0153923]
[2.0153923]
[2.0153923]
[2.01495798]
[2.01495798]
[2.01495798]
[2.01468999]
[2.01557194]
[2.01557194]
[2.01557194]
[2.01512431]
[2.01374643]
[2.01374643]
[2.01374643]
[2.01374643]
[2.01374643]
[2.01374643]
[2.01374643]
[2.01374643]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.0133238]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01322779]
[2.01

KeyboardInterrupt: 